In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE139384"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE139384"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE139384.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE139384.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE139384.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Synaptopathy in Kii ALS/PDC, a disease concept based on transcriptome analyses of human brains"
!Series_summary	"Amyotrophic lateral sclerosis (ALS) and parkinsonism-dementia complex (PDC) (ALS/PDC) is a unique endemic neurodegenerative disease, with high-incidence foci in the Kii Peninsula, Japan. Although ALS/PDC presents with multiple proteinopathies, the genetic and environmental factors that influence disease onset remain unknown. We performed transcriptome analyses of patients’ brains, which may provide new insights into the pathomechanisms underlying Kii ALS/PDC."
!Series_summary	"We prepared frozen brains from 3 healthy controls (frontal lobe and temporal lobe), 3 patients with Alzheimer’s disease (AD) (frontal lobe and temporal lobe) as tauopathy-disease controls, and 21 patients with Kii ALS/PDC (frontal lobe and/or temporal lobe). We acquired microarray data from the cerebral gray and white matter tissues of Kii ALS/PDC patients."
!Seri

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Callable, Optional, Dict, Any
import os
import json

# 1. Gene Expression Data Availability
# Based on the background information, it mentions "transcriptome analyses" and use of 
# "HumanHT-12 v4 Expression BeadChip" which indicates gene expression data
is_gene_available = True

# 2.1 Data Availability for trait, age, and gender
# From the Sample Characteristics Dictionary
# For trait: Looking at rows 0 and 1, row 1 has 'clinical phenotypes' including healthy control and AD
trait_row = 1

# For age: Looking at rows 2 and 3, both contain age information
age_row = 2

# For gender: Row 1 and 2 contain gender information
gender_row = 1

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert trait value to binary (0: Control, 1: Alzheimer's Disease)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if "Healthy Control" in value:
        return 0
    elif "Alzheimer" in value:
        return 1
    else:
        # Other clinical phenotypes like ALS, PDC, etc. are not relevant for Alzheimer's Disease study
        return None

def convert_age(value: str) -> float:
    """Convert age value to continuous numeric value"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> int:
    """Convert gender value to binary (0: Female, 1: Male)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if "Female" in value:
        return 0
    elif "Male" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Save initial validation results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Use the sample characteristics from previous step instead of trying to read from a file
    # Create a DataFrame from the sample characteristics dictionary provided in the previous output
    sample_chars = {
        0: ['subject id: CT1', 'subject id: CT2', 'subject id: CT3', 'subject id: AD1', 'subject id: AD2', 'subject id: AD3', 'clinical phenotypes: ALS', 'clinical phenotypes: ALS+D', 'clinical phenotypes: PDC+A', 'clinical phenotypes: PDC'],
        1: ['clinical phenotypes: Healthy Control', 'clinical phenotypes: Alzheimer`s Disease', 'gender: Female', 'gender: Male'],
        2: ['gender: Male', 'age: 66', 'age: 77', 'age: 70', 'age: 74', 'age: 76', 'age: 60', 'age: 79', 'age: 71', 'age: 63', 'age: 65', 'age: 81', 'age: 73', 'age: 72', 'age: 75', 'age: 85'],
        3: ['age: 75', 'age: 76', 'age: 83', 'age: 84', 'age: 87', 'age: 88', 'age: 67', 'age: 68', 'age: 86', 'age: 74', 'tissue: Human Postmortem Brain'],
        4: ['tissue: Human Postmortem Brain', 'tissue subtype: Frontal lobe', 'tissue subtype: Temporal lobe'],
        5: ['tissue subtype: Frontal lobe', 'tissue subtype: Temporal lobe', None]
    }
    
    # Convert the dictionary to DataFrame format needed for geo_select_clinical_features
    clinical_data = pd.DataFrame.from_dict(sample_chars, orient='index')
    
    try:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical data:", preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error during clinical feature extraction: {e}")


Preview of selected clinical data: {0: [0.0, nan, nan], 1: [1.0, 66.0, nan], 2: [nan, 77.0, 0.0], 3: [nan, 70.0, 1.0], 4: [nan, 74.0, nan], 5: [nan, 76.0, nan], 6: [nan, 60.0, nan], 7: [nan, 79.0, nan], 8: [nan, 71.0, nan], 9: [nan, 63.0, nan], 10: [nan, 65.0, nan], 11: [nan, 81.0, nan], 12: [nan, 73.0, nan], 13: [nan, 72.0, nan], 14: [nan, 75.0, nan], 15: [nan, 85.0, nan]}
Clinical data saved to ../../output/preprocess/Alzheimers_Disease/clinical_data/GSE139384.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651254', 'ILMN_1651262', 'ILMN_1651315', 'ILMN_1651354',
       'ILMN_1651385', 'ILMN_1651405', 'ILMN_1651429', 'ILMN_1651438',
       'ILMN_1651498', 'ILMN_1651680', 'ILMN_1651705', 'ILMN_1651719',
       'ILMN_1651735', 'ILMN_1651745', 'ILMN_1651799', 'ILMN_1651819'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# The identifiers shown start with "ILMN_" which indicates Illumina probe IDs
# These are not standard human gene symbols but probe identifiers from Illumina microarray platforms
# They need to be mapped to proper gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. First identify which columns in gene_annotation contain probe IDs and gene symbols
# From previewing gene_annotation, we can see:
# - 'ID' column contains Illumina probe IDs (ILMN_*) which match our gene expression data
# - 'Symbol' column contains gene symbols

# 2. Get a gene mapping dataframe with the ID and Symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Preview the mapping dataframe
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
# This handles many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene-level expression data
print("Gene-level expression data preview:")
print(gene_data.shape)
print("First 10 gene symbols:")
print(gene_data.index[:10])


Gene mapping preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}
Gene-level expression data preview:
(5550, 33)
First 10 gene symbols:
Index(['A2BP1', 'A2M', 'AADACL1', 'AADAT', 'AAGAB', 'AARS', 'AARSD1',
       'AASDHPPT', 'AATK', 'ABAT'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (5434, 33)
Normalized gene data saved to ../../output/preprocess/Alzheimers_Disease/gene_data/GSE139384.csv
Loading the original clinical data...


Extracting clinical features...
Clinical data preview:
{'GSM4140293': [0.0, nan, nan], 'GSM4140294': [0.0, nan, nan], 'GSM4140295': [0.0, nan, nan], 'GSM4140296': [0.0, nan, nan], 'GSM4140297': [0.0, nan, nan], 'GSM4140298': [0.0, nan, nan], 'GSM4140299': [1.0, nan, nan], 'GSM4140300': [1.0, nan, nan], 'GSM4140301': [1.0, nan, nan], 'GSM4140302': [1.0, nan, nan], 'GSM4140303': [1.0, nan, nan], 'GSM4140304': [1.0, nan, nan], 'GSM4140305': [nan, 66.0, 0.0], 'GSM4140306': [nan, 77.0, 1.0], 'GSM4140307': [nan, 70.0, 0.0], 'GSM4140308': [nan, 74.0, 0.0], 'GSM4140309': [nan, 76.0, 0.0], 'GSM4140310': [nan, 60.0, 0.0], 'GSM4140311': [nan, 79.0, 1.0], 'GSM4140312': [nan, 71.0, 0.0], 'GSM4140313': [nan, 63.0, 0.0], 'GSM4140314': [nan, 65.0, 1.0], 'GSM4140315': [nan, 70.0, 0.0], 'GSM4140316': [nan, 81.0, 0.0], 'GSM4140317': [nan, 70.0, 0.0], 'GSM4140318': [nan, 74.0, 1.0], 'GSM4140319': [nan, 73.0, 0.0], 'GSM4140320': [nan, 72.0, 1.0], 'GSM4140321': [nan, 72.0, 0.0], 'GSM4140322': [nan, 75.0, 1.

Linked data shape after handling missing values: (12, 5436)
Checking for bias in trait distribution...
For the feature 'Alzheimers_Disease', the least common label is '0.0' with 6 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Alzheimers_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Age' in this dataset is fine.

Dataset usability: True
Linked data saved to ../../output/preprocess/Alzheimers_Disease/GSE139384.csv
